In [52]:
import pandas as pd
import numpy as np

# --
# Read Data
# --

In [53]:
data = pd.read_csv("datasets/weatherAUS.csv")
# data.head()

# --
# Data Preprocessing
# --

In [55]:
# data = np.concatenate((X, np.reshape(y, (-1, 1))), axis=1)
t = (data.dtypes == "float64")
num_cols = list(t[t].index)

# Ada Berapa Data Null di Column Numeric?
totalNull = 0
for i in num_cols:
    # print(i, data[i].isnull().sum())
    break
    
# Fill NA
for i in num_cols:
    data[i].fillna(data[i].median(), inplace=True)
    
# data.info()

In [56]:
# Date
# def encode(data, col, max_val):
#     data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
#     data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)

#     return data

lengths = data["Date"].str.len()
lengths.value_counts()

data['Date']= pd.to_datetime(data["Date"])
data['year'] = data.Date.dt.year
data['month'] = data.Date.dt.month
# data = encode(data, 'month', 12)
data['day'] = data.Date.dt.day
# data = encode(data, 'day', 31)

# data.head()

In [57]:
# Label Encoder
from sklearn.preprocessing import LabelEncoder

# Dapatkan kolom categoricacl
s = (data.dtypes == "object")
object_cols = list(s[s].index)

# Lengkapi kolom yang null
for i in object_cols:
    data[i].fillna(data[i].mode()[0], inplace=True)

# Label Encoder
label_encoder = LabelEncoder()
for i in object_cols:
    data[i] = label_encoder.fit_transform(data[i])

data.head()

Categorical variables:
['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  int32         
 2   MinTemp        145460 non-null  float64       
 3   MaxTemp        145460 non-null  float64       
 4   Rainfall       145460 non-null  float64       
 5   Evaporation    145460 non-null  float64       
 6   Sunshine       145460 non-null  float64       
 7   WindGustDir    145460 non-null  int32         
 8   WindGustSpeed  145460 non-null  float64       
 9   WindDir9am     145460 non-null  int32         
 10  WindDir3pm     145460 non-null  int32         
 11  WindSpeed9am   145460 non-null  float64       
 12  WindSpeed3pm   145460 non-null  float64       
 

In [19]:
dataNP = data.to_numpy()
X = dataNP[:, 0:22]
y = dataNP[:, 22]

In [20]:
# Splitting Training and Testing
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = (train_test_split(X, y,test_size = 0.4,random_state=1))

print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(87276, 22)
(58184, 22)
(87276,)
(58184,)


In [14]:
# IQR Detecting Outlier
dataOut = np.concatenate((xtrain, np.reshape(ytrain, (-1, 1))), axis=1)
outliner = 0
for out in range(22):
  colData = dataOut[:,out]
  q1,q3 = np.percentile(colData,[25,75])
  iqr = q3-q1
  lb,ub = q1-1.5*iqr, q3+1.5*iqr

  outliers = dataOut[np.where((colData < lb) | (colData > ub))]
  outliner += len(outliers)

  print("Outliers in Column ", out, " is ", len(outliers))

  dataOut = dataOut[np.where((colData >= lb) & (colData <= ub))]

print("Detected total = ", outliner, " outliers")

pd.DataFrame(data)

xtrain = data[:,0:22]
ytrain = data[:,22]

print(xtrain.shape)
print(ytrain.shape)

TypeError: can't multiply sequence by non-int of type 'float'

In [21]:
# Standard Deviation Detecting Outlier
data = np.concatenate((xtrain, np.reshape(ytrain, (-1, 1))), axis=1)

for idx in range(0, 22):
  col_data = data[:, idx]
  
  mean = col_data.mean()
  std = col_data.std()
  lb, ub = mean-(3*std), mean+(3*std)

  data = data[np.where((col_data >= lb) & (col_data <= ub))]

xtrain = data[:, 0:22]
ytrain = data[:, 22]

print(data.shape)
print(xtrain.shape)
print(ytrain.shape)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(original_data)
original_data = scaler.transform(original_data)

In [ ]:
# MinMax Scaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(original_data)
original_data = scaler.transform(original_data)